In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:

dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
data = pd.get_dummies(dataset)
data.head()

In [ ]:
features = data.drop(['expenses'], axis=1)
labels = data['expenses']
features.head()

In [ ]:
train_dataset, test_dataset, train_labels, test_labels = train_test_split(features, labels, random_state=42, test_size=0.2)
print("train", train_dataset.shape, train_labels.shape)
print("test", test_dataset.shape, test_labels.shape)

In [ ]:
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')
train_dataset = ct.fit_transform(train_dataset)
test_dataset = ct.transform(test_dataset)

In [ ]:
model = keras.models.Sequential()
input = layers.InputLayer(input_shape=(features.shape[1],))
model.add(input)
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mse', metrics=['mae','mse'], optimizer=opt)

In [ ]:
model.fit(train_dataset, train_labels, epochs = 20, batch_size = 5, verbose = 0)

In [ ]:

loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
    print("You passed the challenge. Great job!")
else:
    print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
